# This notebook shows off Background options in views_mapper2
Ensure that the views_mapper2 is up to date with the latest update as documented in main read me file
Things covered in this notebook are background options, transparency, masking, and views_experimental labels

In [ ]:
#mapper version verification
!pip show views_mapper2

In [ ]:
#obtain data, country month
from viewser import Queryset, Column

queryset = (Queryset("Mapper_tester_cm", "country_month")
            # CONFLICT TYPES NOT LOGGED and NOT time lagged
            .with_column(Column("ged_best_sb", from_table="ged2_cm", from_column="ged_sb_best_sum_nokgi")
                         .transform.missing.fill()
                         )
            # .with_column(Column('name', from_table='country', from_column= 'name')
            # the above does not seem to want to work

            .with_theme("fatalities")

            .describe("""Fatalities conflict history, cm level
            Mapper testing

            """)

            )

cm_master = queryset.publish().fetch()

In [ ]:
#fetch the multipolygons for mapping
import geopandas as gpd
import sqlalchemy as sa
from ingester3.config import source_db_path

engine = sa.create_engine(source_db_path)
gdf_ci_master = gpd.GeoDataFrame.from_postgis(
    "SELECT id as country_id, in_africa, name, in_me, geom FROM prod.country",
    engine,
    geom_col='geom'
)
gdf_ci_master = gdf_ci_master.to_crs(4326)

In [ ]:
#Mapper2 
from views_mapper2.mapper2 import *
from views_mapper2.label_writer import *
from views_mapper2.dictionary_writer import *
from views_mapper2.BBoxWriter import*

# showing off the various backgrounds

background tiles are essentially jpeg like images that vary depending on the location where we focus. More information on regional information with closer look and less further out. As we are using them in their non-native forms through geopandas, a lot of the original html/web modifiability is lost. So not possible to ask only for country names and large cities. 

To see examples, please substitute with the names below in the name variable, e.g. name = 'OpenTopoMap'

Some of the maps look better as background in masked versions. 


1. OpenTopoMap - includes both country names and topography, colorful, no built in layered labels so 100% opacity will prevent the view of the country names. Additionally colorful background interferes with the 1:1 use of the colorbar when violence layer is transparent as one must account for underlying color and how it interacts with the transparent overlay. But the general gist of the color is still understandble. Works well for masked versions of specific countries.
2. OpenStreetMap - includes names of countries, regions, etc. No topographic data. No build in layered labels so 100% opacity will prevent the view of country names. Monochromatic colorpalette allows for easier interpretation of transparent layer overlayers. 
3. StamenLite - includes names of countries, regions, etc. No topographic data. Greyscale. Built layer. Does not look good for 100% opacity but this version will still make it possible to see the labels. Due to nature of geopandas, labels occasionally collide with the viewtextbox as they are layered on top of everything. 
4. StamenWatercolor - same as above but with colorful theme, no country borders build in.
5. StamenTerrain - tiles with both country names and terrain, no built in labels so 100% opacity will interfere.
6. StamenTerrainBackground - only terrain information of the above
7. CartoDBPositron - similar to StamenLite but with slightly less information
8. CartoDBVoyager - similar to above, more recent release


In [ ]:
data= cm_master.copy()
gdf = gdf_ci_master.copy()

data = data.join(gdf.set_index("country_id"))
gdf = gpd.GeoDataFrame(data, geometry="geom")

#adjust the background here
name = 'CartoDBPositron'


a = Mapper2(
    width=10,
    height=10,
    frame_on=True,
    title=name, 
    bbox = bbox_from_cid('ame')
).add_layer(
    gdf=gdf.loc[450],
    map_dictionary = dictionary_stand,
    cmap = 'rainbow',
    transparency = 1,
    edgecolor="black",
    linewidth=0.7,
    column="ged_best_sb", 
    background = name
).add_views_textbox(
    text='hello world, hello world, hello world',
    textsize=10)

a2 = Mapper2(
    width=10,
    height=10,
    frame_on=True,
    title=f'{name}_country', 
    bbox = bbox_from_cid(79)
).add_layer(
    gdf=gdf.loc[450],
    map_dictionary = dictionary_stand,
    cmap = 'rainbow',
    transparency = 1,
    edgecolor="black",
    linewidth=0.7,
    column="ged_best_sb", 
    background = name
).add_views_textbox(
    text='hello world, hello world, hello world',
    textsize=10)

b = Mapper2(
    width=10,
    height=10,
    frame_on=True,
    title=f'{name}_with_transparency', 
    bbox = bbox_from_cid('ame')
).add_layer(
    gdf=gdf.loc[450],
    map_dictionary = dictionary_stand,
    cmap = 'rainbow',
    transparency = 0.5,
    edgecolor="black",
    linewidth=0.7,
    column="ged_best_sb", 
    background = name
).add_views_textbox(
    text='hello world, hello world, hello world',
    textsize=10)

c = Mapper2(
    width=10,
    height=10,
    frame_on=True,
    title= f'{name}_only', 
    bbox = bbox_from_cid('ame')
).add_layer(
    gdf=gdf.loc[450],
    map_dictionary = dictionary_stand,
    cmap = 'rainbow',
    transparency = 0.0,
    edgecolor="black",
    linewidth=0.7,
    column="ged_best_sb", 
    background = name
).add_views_textbox(
    text='hello world, hello world, hello world',
    textsize=10)

# Experimental labels

experimental labels are my own creation. It creates and centers a name in the country geometry
works at variable success rate
adjustment of the font size can result in improved results
recommend with zoom in on specific countries

adjust font to note how sometimes labels work and other times they start to collide

In [ ]:
data= cm_master.copy()
gdf = gdf_ci_master.copy()

data = data.join(gdf.set_index("country_id"))
gdf = gpd.GeoDataFrame(data, geometry="geom")

font = 10


a = Mapper2(
    width=10,
    height=10,
    frame_on=True,
    title='views_experimental1', 
    bbox = bbox_from_cid('ame')
).add_layer(
    gdf=gdf.loc[450],
    map_dictionary = dictionary_stand,
    cmap = 'rainbow',
    transparency = 1,
    edgecolor="black",
    linewidth=0.7,
    column="ged_best_sb",
    views_experimental_labels = 10
).add_views_textbox(
    text='hello world, hello world, hello world',
    textsize=10)

a2 = Mapper2(
    width=10,
    height=10,
    frame_on=True,
    title= 'views_experimental2', 
    bbox = bbox_from_cid(79)
).add_layer(
    gdf=gdf.loc[450],
    map_dictionary = dictionary_stand,
    cmap = 'rainbow',
    transparency = 1,
    edgecolor="black",
    linewidth=0.7,
    column="ged_best_sb",
    views_experimental_labels = font
).add_views_textbox(
    text='hello world, hello world, hello world',
    textsize=10)

## Masks

Masks allow for greying out of the regions not of interest
The available sections are africa, africa+middle east, and country
All must be written as strings and require a column with country_names
I have defaulted it to look for names as written in the name of the country table (some variation from iso_names)
Ensure appropriate spelling of the country before use

Masks can be combined with the backgrounds as underlying layers or used on their own
Below are two cells that allow you to use either method
Additionally a third cell provides a demonstration for pg level maps

In [ ]:
#Masking using layers, this version of masking is required if you wish to use no background
#essentially you create a layer where all values are grey-scaled and placing a mask over the top
#changing transparency of the base layer can help achieve a more or less washed out appearance
#WARNING: very northern or southern countries start to behave strangely using 2 layers
#use the add_mask on its own via second method for those countries as globe bbox

data= cm_master.copy()
gdf = gdf_ci_master.copy()

data = data.join(gdf.set_index("country_id"))
gdf = gpd.GeoDataFrame(data, geometry="geom")

#you can adjust these to see various effects and combinations
background2 = None #adjust this and note no difference, as the layer1 obscurs the background at transparency = 1
views_label_font = 7
textbox_label_font = 20
masking_location_choice = "Algeria" #choices allowed are 'africa', 'ame', or country_name


ame=Mapper2(
    width=20,
    height=20,
    frame_on=True,
    title='mask with layers',
    bbox=bbox_from_cid('ame')
).add_layer(
    gdf=gdf.loc[450],
    map_dictionary = dictionary_stand,
    cmap = 'binary',
    transparency = 1.0,
    edgecolor="black",
    linewidth=0.5,
    column="ged_best_sb"
).add_mask(
    gdf = gdf.loc[450],
    map_dictionary = dictionary_stand,
    cmap = 'rainbow',
    transparency = 1,
    masking_location = masking_location_choice,
    views_experimental_labels = views_label_font,
    column='ged_best_sb',
    background = background2,
    edgecolor="black",
    linewidth=1
).add_views_textbox(
    text='hello world, hello world',
    textsize=textbox_label_font)

In [ ]:
#Masking by using .add_mask requires a background
#this version of masking requires the use of background, otherwise you will create floating regions

data= cm_master.copy()
gdf = gdf_ci_master.copy()

data = data.join(gdf.set_index("country_id"))
gdf = gpd.GeoDataFrame(data, geometry="geom")

#you can adjust these to see various effects and combinations
#you can also delete the views_experimental_labels to see appearance without them
#avoid using the views_labels with globe level
background2 = 'StamenTerrainBackground' #use None to see the need for background use
views_label_font = 7
textbox_label_font = 20
masking_location_choice = 'Congo' #choices allowed are 'africa', 'ame', or country_id number


ame=Mapper2(
    width=20,
    height=20,
    frame_on=True,
    title='mask with layers',
    bbox=bbox_from_cid('ame') ##you can change this to ame, africa or country bbox
).add_mask(
    gdf = gdf.loc[450],
    map_dictionary = dictionary_stand,
    cmap = 'rainbow',
    transparency = 1,
    masking_location = masking_location_choice,
    column='ged_best_sb',
    background = background2,
    views_experimental_labels = views_label_font,
    edgecolor="black",
    linewidth=1
).add_views_textbox(
    text='hello world, hello world',
    textsize=textbox_label_font)

### PG level masking, note the slightly more complicated geometry creation
- also note that the information for pg is not available for the entire globe

In [ ]:
#PG level mapping example with masking


#Data gathering
queryset = (Queryset("m_fatalities_conflict_pgm", "priogrid_month")
              #CONFLICT TYPES NOT LOGGED and NOT time lagged 
            .with_column(Column("ged_best_sb", from_table = "ged2_pgm", from_column = "ged_sb_best_sum_nokgi")
                         .transform.missing.fill()
                        )
                
            #.with_column(Column('name', from_table='country', from_column= 'name')
            #the above does not seem to want to work 
                         
            .with_theme("fatalities")
                         
            .describe("""Fatalities conflict history, pgm level
            Fatalities description 
                
            """)
           )

pg_master = queryset.publish().fetch()


#fetching geometries
engine = sa.create_engine(source_db_path)
gdf_pid = gpd.GeoDataFrame.from_postgis(
    "SELECT id as priogrid_gid, in_africa, in_me, geom FROM prod.priogrid", 
    engine, 
    geom_col='geom'
)
gdf_pid = gdf_pid.to_crs(4326)

#now grab the country names
pg_country_queryset = (Queryset("m_associate_country_2_priogrid", "priogrid_month")
              #CONFLICT TYPES NOT LOGGED and NOT time lagged 
            #.with_column(Column("month_id", from_table = "priogrid_month", from_column = "month_id"))
            .with_column(Column('name', from_table = 'country', from_column = 'name'))
                             
            .with_theme("Mapping")
                         
            .describe("""Mapping, allows for association of priogrid with country 
                
            """)
            
           )
pg_country = pg_country_queryset.publish().fetch().reset_index()
#pg_country_cleaned = pg_country.reset_index().drop(['month_id'], axis = 1).drop_duplicates(subset=['priogrid_gid'])

gdf_pid_master = pd.merge(left = gdf_pid, right = pg_country, left_on = 'priogrid_gid', 
                          right_on = 'priogrid_gid', how = 'right').set_index(['month_id', 'priogrid_gid'])


#grab country geometries for borders creation
gdf_ci_master = gpd.GeoDataFrame.from_postgis(
    "SELECT id as country_id, in_africa, in_me, geom FROM prod.country",
    engine,
    geom_col='geom'
)
gdf_ci_master = gdf_ci_master.to_crs(4326)

In [ ]:
#mapping code for the above

data = pg_master.copy()
gdf = gdf_pid_master.copy()
data_pid = pd.merge(left = data, right = gdf, left_index = True, right_index = True, how = 'left')
gdf = gpd.GeoDataFrame(data_pid, geometry="geom")
gdf_c = gdf_ci_master.copy()

#you can adjust these to see various effects and combinations
#please do not use the experimental labels as the name will be the name of priogrid_cell
background2 = 'StamenTerrainBackground' #use None to see the need for background use
textbox_label_font = 20
masking_location_choice = 'Somalia' #choices allowed are 'africa', 'ame', or country_id number


masked=Mapper2(
    width=20,
    height=20,
    frame_on=True,
    title='mask with layers',
    bbox=bbox_from_cid('ame')
).add_mask(
    gdf = gdf.loc[450],
    map_dictionary = dictionary_stand,
    cmap = 'rainbow',
    transparency = 1,
    masking_location = masking_location_choice,
    column='ged_best_sb',
    background = background2,
    edgecolor="black",
    linewidth=0.5
).add_views_textbox(
    text='hello world, hello world',
    textsize=textbox_label_font)

#Jim procedure for country border creation using the geometries inherent to views3
ax = masked.ax
masked_with_borders=gdf_c.plot(ax=ax,edgecolor='black',linewidth=2.0,facecolor='None')